Defining an "adopted user" as a user who has logged into the product on three separate days in at least one seven­day period, identify which factors predict future user adoption.

We suggest spending 1-2 hours on this, but you're welcome to spend more or less. Please send us a brief writeup of your findings (the more concise, the better - no more than one page), along with any summary tables, graphs, code, or queries that can help us understand your approach. Please note any factors you considered or investigation you did, even if they did not pan out. Feel free to identify any further research or data you think would be valuable.

# Import and read csv files

In [1]:
import datetime

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
takehome_users = pd.read_csv('./takehome_users.csv',encoding="ISO-8859-1")
takehome_user_engagement = pd.read_csv('./takehome_user_engagement.csv',parse_dates=["time_stamp"])

A user table ("takehome_users") with data on 12,000 users who signed up for the product in the last two years. This table includes:

* name: the user's name

* object_id: the user's id

* email: email address

* creation_source: how their account was created. This takes on one of 5 values: 
    * PERSONAL_PROJECTS: invited to join another user's personal workspace 
    * GUEST_INVITE: invited to an organization as a guest (limited permissions) 
    * ORG_INVITE: invited to an organization (as a full member) 
    * SIGNUP: signed up via the website 
    * SIGNUP_GOOGLE_AUTH: signed up using Google Authentication (using a Google email account for their login id) 
* creation_time: when they created their account 
* last_session_creation_time: unix timestamp of last login 
* opted_in_to_mailing_list: whether they have opted into receiving marketing emails 
* enabled_for_marketing_drip: whether they are on the regular marketing email drip 
* org_id: the organization (group of users) they belong to 
* invited_by_user_id: which user invited them to join (if applicable).

# Data analysis and cleaning

In [3]:
# Visualizing the takehome_users dataframe
takehome_users.head(10)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0
5,6,2013-12-17 03:37:06,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,1.387424e+09,0,0,197,11241.0
6,7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,1.356010e+09,0,1,37,NaN
7,8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,NaN,1,1,74,NaN
8,9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,302,NaN
9,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1.401833e+09,1,1,318,4143.0


In [4]:
# Summary of data in table - we notice that there seem to be null values in the 'last_session_creation_time' column a

takehome_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [5]:
len(takehome_users['object_id'].unique())

12000

In [6]:
len(takehome_users['email'].unique())

11980

A usage summary table ("takehome_user_engagement") that has a row for each day that a user logged into the product.

In [7]:
# Sort the table by user id and timestamp of their logins in chronological order
takehome_user_engagement.sort_values(by = ['user_id', 'time_stamp']).head(100)
takehome_user_engagement.head(10)

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1
5,2013-12-31 03:45:04,2,1
6,2014-01-08 03:45:04,2,1
7,2014-02-03 03:45:04,2,1
8,2014-02-08 03:45:04,2,1
9,2014-02-09 03:45:04,2,1


In [8]:
len(takehome_user_engagement['user_id'].unique())

8823

In [9]:
# View the null columns of the takehome users data frame
takehome_users[takehome_users.last_session_creation_time.isnull()]

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
7,8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,NaN,1,1,74,NaN
8,9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,302,NaN
11,12,2014-04-17 23:48:38,Mathiesen Lærke,LaerkeLMathiesen@cuvox.de,ORG_INVITE,NaN,0,0,130,9270.0
14,15,2013-07-16 21:33:54,Theiss Ralf,RalfTheiss@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,175,NaN
15,16,2013-02-11 10:09:50,Engel René,ReneEngel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,211,NaN
17,18,2013-08-24 00:26:46,Gregersen Celina,CelinaAGregersen@jourrapide.com,GUEST_INVITE,NaN,0,0,2,3153.0
25,26,2013-07-28 00:53:32,Wexler Florian,FlorianWexler@jourrapide.com,ORG_INVITE,NaN,0,0,56,11857.0
31,32,2013-04-17 16:21:30,Perez Timothy,TimothyJPerez@yahoo.com,PERSONAL_PROJECTS,NaN,1,1,98,NaN
33,34,2014-02-03 20:55:18,Holm Sara,SaraMHolm@hotmail.com,ORG_INVITE,NaN,0,0,33,3721.0
34,35,2013-08-01 10:55:26,Krueger Dieter,DieterKrueger@gustr.com,PERSONAL_PROJECTS,NaN,0,0,59,NaN


In [10]:
# Noticed null values so dropped the rows with null values
takehome_users = takehome_users.dropna(subset=['last_session_creation_time'])
takehome_users.head(10)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0
5,6,2013-12-17 03:37:06,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,1.387424e+09,0,0,197,11241.0
6,7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,1.356010e+09,0,1,37,NaN
9,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1.401833e+09,1,1,318,4143.0
10,11,2013-12-26 03:55:54,Paulsen Malthe,MaltheAPaulsen@gustr.com,SIGNUP,1.388117e+09,0,0,69,NaN
12,13,2014-03-30 16:19:38,Fry Alexander,AlexanderDFry@cuvox.de,ORG_INVITE,1.396196e+09,0,0,254,11204.0


In [11]:
# Review data in table - no null values
takehome_user_engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null datetime64[ns]
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 MB


# Identifying Adopted Users

In [12]:
#Perform floor operation on the data to the specified freq of days, and convert from datetime to int
takehome_user_engagement['time_stamp'] = takehome_user_engagement['time_stamp'].dt.floor('d').astype(np.int64)

#sorting and remove duplicated days per users 
#takehome_user_engagement = takehome_user_engagement.sort_values(['user_id', 'time_stamp']).drop_duplicates()

In [13]:
#groupby user_id and provide rolling window calculations.
a = takehome_user_engagement.groupby('user_id')['time_stamp'].rolling(window=3)

b = pd.to_timedelta((a.max()- a.min())).dt.days

In [14]:
print(b[:10])

user_id   
1        0     NaN
2        1     NaN
         2     NaN
         3    24.0
         4    26.0
         5    22.0
         6    14.0
         7    34.0
         8    31.0
         9     6.0
Name: time_stamp, dtype: float64


In [15]:
adopted_users = b[b == 7].index.get_level_values('user_id').tolist()
len(adopted_users)

6677

In [16]:
c = list(set(adopted_users))
c.sort()

In [17]:
takehome_users['adopted_user'] = 0
for val in c:
    takehome_users.loc[takehome_users['object_id'] ==val, 'adopted_user'] = 1

In [18]:
# Takehome users dataframe with the populated 'adopter_user' column with a binary response
takehome_users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,1
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,0


# Machine Learning Classification
We choose to use a subset of our features, omitting the features which identify the user such as user ID, name, email, etc. which are not useful for classification. <br>
We use a random forest classifier here as our choice of model and we analyze the predictive power of our different features.

In [19]:
feature_cols = ['creation_source', 'last_session_creation_time', 'enabled_for_marketing_drip', 'org_id']
X = takehome_users.loc[:, feature_cols]
y = takehome_users.adopted_user
print(X.shape, y.shape)

(8823, 4) (8823,)


In [20]:
## Data pre-processing for Random Forest Classifier

In [21]:
# Conversion of categorical variables to ordinal encoded variables
from sklearn import preprocessing
enc = preprocessing.OrdinalEncoder()

X_list = np.array(X['creation_source'])
X_list = [[val] for val in X_list]
enc.fit(X_list)
X['creation_source'] = enc.transform(np.array(X['creation_source']).reshape(-1,1))

# Normalizing the timestamps for last session creation time
max_time = max(X.last_session_creation_time)
min_time = min(X.last_session_creation_time)

X.last_session_creation_time = X.last_session_creation_time.apply(lambda x: (x - min_time)/(max_time-min_time))

X.head()

,creation_source,last_session_creation_time,enabled_for_marketing_drip,org_id
0,0.0,0.938254,0,11
1,1.0,0.908366,0,1
2,1.0,0.397434,0,94
3,0.0,0.483504,0,1
4,0.0,0.320639,0,193


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# Fitting the Random Forest Classifier model
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(X_train, y_train)

C:\Users\JLH2TCQ\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [34]:
# Performance Metrics of our model
from sklearn.metrics import confusion_matrix
y_pred = rf.predict(X_test)
for i in range(len(y_pred)):
    if y_pred[i] >= 0.5:
        y_pred[i] = 1
    else:
        y_pred[i] = 0
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
precision = tp / (tp + fp)
recall = tp / (tp + fn)
F1 = 2*precision*recall/(precision + recall)
print('Confusion Matrix:', '\n', cm)
print('Precision: \t', precision)
print('Recall: \t', recall)
print('F1 Score: \t', F1)

Confusion Matrix: 
 [[1397   80]
 [ 119  169]]
Precision: 	 0.678714859437751
Recall: 	 0.5868055555555556
F1 Score: 	 0.6294227188081937


In [25]:
# Looking at importance of the features we used for classification
print("Features sorted by their score:")
print(sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), feature_cols), 
             reverse=True))

Features sorted by their score:
[(0.7447, 'last_session_creation_time'), (0.1946, 'org_id'), (0.046, 'creation_source'), (0.0146, 'enabled_for_marketing_drip')]
